<a href="https://colab.research.google.com/github/kjfms/NIA-Project/blob/main/Code/%EC%9E%A5%ED%9D%A5%EA%B5%B0%EC%B2%AD_%EB%AF%BC%EC%9B%90%EB%B4%89%EC%82%AC%EA%B3%BC_%EB%AF%BC%EC%9B%90%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Konlpy 설치

In [ ]:
# 코랩에서 한글 깨질 때 실행
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

In [ ]:
!apt-get update !apt-get install g++ openjdk-8-jdk python-dev python3-dev 
!pip3 install JPype1-py3 
!pip3 install konlpy 
!JAVA_HOME="C:\Program Files\Java\jdk-16.0.2"

In [ ]:
%%bash
bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)
pip3 install /tmp/mecab-python-0.996

### 1. 필요 라이브러리 import

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import gensim

In [ ]:
data = pd.read_excel('/content/drive/MyDrive/장흥군청/청구서전체현황(2020-01-01_2020-12-31).xlsx')

In [ ]:
import konlpy
from konlpy.tag import Kkma, Komoran, Hannanum, Okt
from konlpy.utils import pprint
from konlpy.tag import Mecab


In [ ]:
mecab = Mecab()
okt = Okt()

In [ ]:
from matplotlib import font_manager, rc
import matplotlib.font_manager as fm 
from wordcloud import WordCloud

from collections import Counter

In [ ]:
data[data['처리부서'].str.contains('민원봉사과', na = True)]['청구제목']

In [ ]:
# 첨부파일이 있는 민원들의 행번호를 리스트로 정리
contents_null_index  = [i for i in range(len(data)) if data.loc[i,'접수번호'] in contents_null]

In [ ]:
data.fillna({'공개내용' : ' '},inplace=True)

In [ ]:
data.fillna(' ', inplace = True)

In [ ]:
data.isnull().sum()

## 처리부서 : 민원봉사과인 민원만 추출

In [ ]:
contents_null = [6343850,6342663,6347859,6361980, 6360414,6365844,6370809,6376187, 6376127,6375886,6392497, 6408176 ,6411220 ,6424017 ,6423993, 6421206, 6429766, 6433720, 6433613, 6470149,
6477407, 6482634,6486872,6486866,6484838,6496572,6501522,6501474,6504238,6513102,6511367,6511128,6521753,6511115,6533115,6552564, 6564301,6563158, 6568608, 6580512, 6587557, 6592447, 6599511,
6599500, 6599471, 6597479, 6597189, 6645357, 6645357, 6659233, 6663445, 6662997, 6671311, 6670436, 6678598, 6682150, 6696458, 6699212, 6703218, 6717327, 6724342, 6756529, 6772665, 6794693,
6793919, 6817680, 6822431, 6821205, 6833506, 6839221, 6838795, 6843988, 6855427, 6869560, 6907249, 6936585, 7148167, 7288599, 7290360]

In [ ]:
sentence = []

for i in range(len(data)) :
  if data.loc[i,'처리부서'].find('민원봉사과') != -1 :
      if (i in contents_null_index) :
        contents = data.loc[i, '공개내용']
        title = ''
      else :
        contents = data.loc[i, '청구내용']
        title = data.loc[i, '청구제목']

  sentence.append([c for c in okt.nouns(contents) if len(c) >=2 ])
  sentence.append([t for t in okt.nouns(title) if len(t) >=2 ])

##TF-IDF

###1. stopwords 없이 진행

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(analyzer='word',
                             lowercase=False,
                             tokenizer=None,
                             preprocessor=None,
                             min_df=5, # 최소 문서 수 5건
                             ngram_range=(1,1), #Bigram : ngram_range=(1,2)
                             smooth_idf=True
                             #max_features=10000 #메모리문제로 설정,
                             )

In [ ]:
nouns_df = pd.DataFrame({'nouns' : sentence})

In [ ]:
nouns_df.head()

In [ ]:
tfidf_vector2 = tfidf_vectorizer.fit_transform(nouns_df['nouns'].astype('str')) # tf-idf값으로 채워진 DTM생성

In [ ]:
tfidf_scores2 = tfidf_vector2.toarray().sum(axis = 0)

In [ ]:
tfidf_idx2 = np.argsort(-tfidf_scores2) # 1차원 배열 거꾸로 정렬 의미

In [ ]:
tfidf_scores2_2 = tfidf_scores2[tfidf_idx2] # 인덱스

In [ ]:
tfidf_vocab2 = np.array(tfidf_vectorizer.get_feature_names())[tfidf_idx2] # 인덱스에 해당하는 단어 추출

In [ ]:
tfidf_df2 = pd.DataFrame({'단어' : tfidf_vocab2, '점수' : tfidf_scores2_2})

In [ ]:
tfidf_df2.head(50).style.hide_index()

### 2. stopwords 생성

In [ ]:
stopwords = ['아래', '상상', '제안', '까지', '닷컴', '포털', '사이트', '천만', '오아시스', '이벤트', '접수','서울시','서울','특별시',
             '천만상상','파일','첨부','응모','슬로건','공모','공모전','응모전','신청','경우','때문','정도','사항',
                   '해당','겁니다','이것','저것','그것','돋움','신명', '태명', '한컴', '돋움',
                   '동안','거기','저기','여기','대부분','누구','무엇','고딕','만큼','굴림','감사','건지','텐데',
                   '안녕','이번','걸로','수고','겁니까','그간','그건','그때','글쓴이','누가','니다','다면',
                   '뭔가','상상오아시스','하다','이다','되다','같다','궁','자체','서체','정','서','이','을','있다','없다', '체','관련',
                   '생각', '현재', '진행', '사람', '마음', '남산', '내용', '현실','음','막','김','변','조',
                   '오','참','동','지금','주변','대상','부분','요즘','하루','마련','세대','시간','이상','행위',
                   '활동','구분','사실','과정','모습','기간','선정','단지','자신','발생','지역','기대','마련',
                   '장소','모두','부탁','제공','이용','해주','당시','최근','민원','문제','문제점','현황','개선','방안',
                   '문의','답변','일동','요청','담당자','직원','방법','사용','활용','확인','방식','예전']

stopwords.append('자료')
stopwords.append('정보')
stopwords.append('공개')
stopwords.append('청구')
stopwords.append('정보공개')
stopwords.extend(['공동','자료', '번호', '신고','주소','내역', '장흥군','조지훈'])
stopwords.extend(['대한','용도','사업','기관', '전국', '관내','먼저','등록','열람','조회','미만', '국가','이유','발급','제외', '승인', '상황', '허가','나라','여부','연락','참조','포함', '번호', '붙임', '귀하', '참고', '우리', '다음', '공공', '윤은주', '건데', '윤여욱', '조지훈', '기억', '여자', '남자'])

In [ ]:

tfidf_vectorizer3 = TfidfVectorizer(analyzer='word',
                             lowercase=False,
                             tokenizer=None,
                             preprocessor=None,
                             min_df=5, # 최소 문서 수 5건
                             ngram_range=(1,1), #Bigram : ngram_range=(1,2)
                             smooth_idf=True,
                             stop_words = stopwords
                             )

In [ ]:
tfidf_vectorizer3

In [ ]:
tfidf_vector3 = tfidf_vectorizer3.fit_transform(nouns_df['nouns'].astype('str')) # tf-idf값으로 채워진 DTM생성
tfidf_scores3 = tfidf_vector3.toarray().sum(axis = 0)
tfidf_idx3 = np.argsort(-tfidf_scores3) # 1차원 배열 거꾸로 정렬 의미
tfidf_scores3_2 = tfidf_scores3[tfidf_idx3] # 인덱스
tfidf_vocab3 = np.array(tfidf_vectorizer3.get_feature_names())[tfidf_idx3] # 인덱스에 해당하는 단어 추출
tfidf_df3 = pd.DataFrame({'단어' : tfidf_vocab3, '점수' : tfidf_scores3_2})

In [ ]:
tfidf_df3[tfidf_df3['점수']>30].head(10).style.hide_index()

In [ ]:
tfidf_dict3 = {}

for i in range(len(tfidf_df3)):
    tfidf_dict3[tfidf_df3.loc[i,'단어']] = tfidf_df3.loc[i,'점수']

### 워드클라우드 생성

In [ ]:
font_path = r'C:\Users\user\NanumBarunGothic.ttf'
plt.figure(figsize = (14,10))
plt.rc('font', family='NanumBarunGothic')
plt.rc('font', size = 12)


wordcloud = WordCloud(font_path = '/content/drive/MyDrive/장흥군청/BMJUA(주아체).ttf',relative_scaling = 0.2,
                      background_color = 'white', width = 2000,
                      height = 1000,  max_font_size = 200).generate_from_frequencies(tfidf_dict3)

plt.imshow(wordcloud)
plt.axis('off')
#plt.title('장흥군청 민원봉사과 TF-IDF 점수별 키워드 시각화')
plt.show()

## TF-IDF 점수 0.5 이상인 키워드들만 워드클라우드 생성

In [ ]:
tfidf_dict4 = {key: value for key, value in tfidf_dict3.items() if value > 0.5}

In [ ]:
font_path = r'C:\Users\user\NanumBarunGothic.ttf'
plt.figure(figsize = (14,10))
plt.rc('font', family='NanumBarunGothic')
plt.rc('font', size = 12)


wordcloud = WordCloud(font_path = '/content/drive/MyDrive/장흥군청/BMJUA(주아체).ttf',relative_scaling = 0.2,
                      background_color = 'white', width = 2000,
                      height = 1000,  max_font_size = 200).generate_from_frequencies(tfidf_dict4)

plt.imshow(wordcloud)
plt.axis('off')
#plt.title('장흥군청 민원봉사과 TF-IDF 점수별 키워드 시각화(0.5점 이상)')
plt.show()

---

## 빈도별 키워드 추출

In [ ]:
tmp_df = data[data['처리부서'].str.contains('민원봉사과', na = True)]

## 불용어 사전 생성

In [ ]:
stopwords = ['아래', '상상', '제안', '까지', '닷컴', '포털', '사이트', '천만', '오아시스', '이벤트', '접수','서울시','서울','특별시',
             '천만상상','파일','첨부','응모','슬로건','공모','공모전','응모전','신청','경우','때문','정도','사항',
                   '해당','겁니다','이것','저것','그것','돋움','신명', '태명', '한컴', '돋움',
                   '동안','거기','저기','여기','대부분','누구','무엇','고딕','만큼','굴림','감사','건지','텐데',
                   '안녕','이번','걸로','수고','겁니까','그간','그건','그때','글쓴이','누가','니다','다면',
                   '뭔가','상상오아시스','하다','이다','되다','같다','궁','자체','서체','정','서','이','을','있다','없다', '체','관련',
                   '생각', '현재', '진행', '사람', '마음', '남산', '내용', '현실','음','막','김','변','조',
                   '오','참','동','지금','주변','대상','부분','요즘','하루','마련','세대','시간','이상','행위',
                   '활동','구분','사실','과정','모습','기간','선정','단지','자신','발생','지역','기대','마련',
                   '장소','모두','부탁','제공','이용','해주','당시','최근','민원','문제','문제점','현황','개선','방안',
                   '문의','답변','일동','요청','담당자','직원','방법','사용','활용','확인','방식','예전']

stopwords.append('자료')
stopwords.append('정보')
stopwords.append('공개')
stopwords.append('청구')
stopwords.append('정보공개')
stopwords.append('장흥군')
stopwords.append('공동')
stopwords.append('주소')
stopwords.append('사업')
stopwords.append('전라남도')
stopwords.append('한국')
stopwords.append('내역')
stopwords.append('귀하')
stopwords.append('귀하')
stopwords.extend(['먼저', '목적','다른','업체', '부터', '대수', '대해', '대한', '기관','국가','이유','발급','제외','등록', '승인', '상황', '허가','나라','여부','연락','참조','포함', '번호', '붙임', '귀하', '참고', '우리', '다음', '공공', '윤은주', '건데', '윤여욱', '조지훈', '기억', '여자', '남자'])

## 데이터 전처리

In [ ]:
tmp_df['contents'] = tmp_df.apply(lambda x : x['청구제목']+"\n"+ x['청구내용']+x['공개내용'],axis = 1)

In [ ]:
minwon = tmp_df['contents']

In [ ]:
def cleanText(readData):#텍스트에 포함되어 있는 특수 문자 제거
    text = re.sub(r'[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》\’\“\”\·\n\r\t■◇◆▶;\xa0]', ' ', readData).strip( )
    return text

In [ ]:
import re

In [ ]:
sents = [cleanText(sent) for sent in minwon]  # 문장부호 제거
sents[:1]

In [ ]:
from tqdm import tqdm
import time

tokenized_data = []
for sentence in tqdm(minwon):
    tokenized_sentence = okt.nouns(sentence) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    tokenized_data.append(stopwords_removed_sentence)

In [ ]:
tmp = sum(tokenized_data, [])

In [ ]:
len(tmp)

In [ ]:
unique_word = list(set(tmp))

In [ ]:
len(unique_word)

In [ ]:
unique_word2 = [t for t in unique_word if len(t) >=2 ]

In [ ]:
len(tokenized_data)

## 빈도별 키워드 Count

In [ ]:
cnt = 0
cnt_word = dict()

for i in range(len(unique_word2)) :
  for j in range(len(tokenized_data)) :
     if unique_word2[i] in tokenized_data[j] :
       cnt += 1
  
  k = unique_word2[i]
  cnt_word[k] = cnt
  cnt = 0

In [ ]:
from operator import itemgetter

In [ ]:
dict(sorted(cnt_word.items(), key = itemgetter(1), reverse = True)[:10])

In [ ]:
top10 = dict(sorted(cnt_word.items(), key = itemgetter(1), reverse = True)[:10])

In [ ]:
pd.DataFrame(top10.items(), columns=['단어', '빈도수']).style.hide_index() 

## 워드클라우드

In [ ]:
import matplotlib.pyplot as plt

plt.rc('font', family='NanumBarunGothic') 

In [ ]:
plt.figure(figsize = (14,10))
plt.rc('font', family='NanumBarunGothic')
plt.rc('font', size = 12)


wordcloud = WordCloud(font_path = '/content/drive/MyDrive/장흥군청/BMJUA(주아체).ttf',relative_scaling = 0.2,
                      background_color = 'white', width = 2000,
                      height = 1000,  max_font_size = 200).generate_from_frequencies(cnt_word)

plt.imshow(wordcloud)
plt.axis('off')
#plt.title('장흥군청 민원봉사과 빈도수별 키워드 시각화')
plt.show()

In [ ]:
cnt_word2 = {key: value for key, value in cnt_word.items() if value > 4}

In [ ]:
sorted(cnt_word2.items(), key = itemgetter(1), reverse = True)

In [ ]:
font_path = r'C:\Users\user\NanumBarunGothic.ttf'
plt.figure(figsize = (14,10))
plt.rc('font', family='NanumBarunGothic')
plt.rc('font', size = 12)


wordcloud = WordCloud(font_path = '/content/drive/MyDrive/장흥군청/BMJUA(주아체).ttf',relative_scaling = 0.2,
                      background_color = 'white', width = 2000,
                      height = 1000,  max_font_size = 200).generate_from_frequencies(cnt_word2)

plt.imshow(wordcloud)
plt.axis('off')
#plt.title('장흥군청 민원봉사과 빈도수별 키워드 시각화(빈도수 4이상)')
plt.show()